# Daily Transactions Analysis


In [1]:
# Import and Load your Dataset.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from google.colab import files
uploaded = files.upload()

df = pd.read_csv(list(uploaded.keys())[0])
df.head()

Saving Daily Household Transactions.csv to Daily Household Transactions.csv


,Date,Mode,Category,Subcategory,Note,Amount,Income/Expense,Currency
0,20/09/2018 12:04:08,Cash,Transportation,Train,2 Place 5 to Place 0,30.0,Expense,INR
1,20/09/2018 12:03:15,Cash,Food,snacks,Idli medu Vada mix 2 plates,60.0,Expense,INR
2,19/09/2018,Saving Bank account 1,subscription,Netflix,1 month subscription,199.0,Expense,INR
3,17/09/2018 23:41:17,Saving Bank account 1,subscription,Mobile Service Provider,Data booster pack,19.0,Expense,INR
4,16/09/2018 17:15:08,Cash,Festivals,Ganesh Pujan,Ganesh idol,251.0,Expense,INR


In [2]:
# Data Cleaning
df.isnull().sum()

df['Subcategory'].fillna('Unknown', inplace=True)
df['Note'].fillna('No Description', inplace=True)

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2057 entries, 0 to 2460
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            1303 non-null   datetime64[ns]
 1   Mode            2057 non-null   object        
 2   Category        2057 non-null   object        
 3   Subcategory     2057 non-null   object        
 4   Note            2057 non-null   object        
 5   Amount          2057 non-null   float64       
 6   Income/Expense  2057 non-null   object        
 7   Currency        2057 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 144.6+ KB


/tmp/ipython-input-3326723550.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Subcategory'].fillna('Unknown', inplace=True)
/tmp/ipython-input-3326723550.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [11]:
# Transactions by Payment Mode (Log Scale)
mode_counts = df['Mode'].value_counts().reset_index()
mode_counts.columns = ['Mode', 'Count']
fig = px.bar(mode_counts, x='Mode', y='Count',
             title='Transactions by Payment Mode (Log Scale)',
             color='Mode',
             color_discrete_sequence=px.colors.qualitative.Set2,
             log_y=True,  # Logarithmic scale
             text='Count')
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title='Payment Mode', yaxis_title='Number of Transactions (Log Scale)')
fig.show()

# Top 10 Categories - Horizontal Bar
top_categories = df['Category'].value_counts().nlargest(10).reset_index()
top_categories.columns = ['Category', 'Count']
fig = px.bar(top_categories, x='Count', y='Category',
             orientation='h',
             title='Top 10 Transaction Categories',
             color='Category',
             color_discrete_sequence=px.colors.qualitative.Pastel,
             text='Count')
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title='Number of Transactions', yaxis_title='Category')
fig.show()

# Income vs Expense - Horizontal Bar
income_expense = df['Income/Expense'].value_counts().reset_index()
income_expense.columns = ['Type', 'Count']
fig = px.bar(income_expense, x='Count', y='Type',
             orientation='h',
             title='Income vs Expense Count',
             color='Type',
             color_discrete_sequence=px.colors.diverging.RdBu,
             text='Count')
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title='Number of Transactions', yaxis_title='Type')
fig.show()


In [17]:

# Step 5: Group categories with amount < 20000 into 'Rest Expenses'
category_amount = df.groupby('Category')['Amount'].sum().reset_index()

# Grouping
category_amount['Category'] = category_amount.apply(
    lambda x: 'Rest Expenses' if x['Amount'] < 20000 else x['Category'], axis=1
)

# Regroup to merge all 'Rest Expenses'
category_amount = category_amount.groupby('Category')['Amount'].sum().reset_index()

# Treemap chart
fig = px.treemap(category_amount,
                 path=['Category'],
                 values='Amount',
                 title='Total Amount Spent per Category (Rest Expenses Grouped)',
                 color='Amount',
                 color_continuous_scale='Viridis')

fig.update_traces(textinfo="label+value", hovertemplate='<b>%{label}</b><br>Amount: %{value}')
fig.show()


In [19]:
import plotly.express as px

# Step 6: Time Series Analysis - Interactive Line Chart
daily_trends = df.groupby(df['Date'].dt.date)['Amount'].sum().reset_index()

fig = px.line(daily_trends,
              x='Date',
              y='Amount',
              markers=True,
              title='Daily Transaction Amounts',
              labels={'Amount': 'Total Amount (INR)', 'Date': 'Date'},
              line_shape='spline')  # Smooth curve

fig.update_traces(line=dict(width=2, color='#6b5b95'))
fig.update_layout(
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()


In [21]:
import plotly.express as px

# Step 7: Correlation Heatmap - Top 10 Categories
top_categories = df.groupby('Category')['Amount'].sum().nlargest(10).index
filtered_df = df[df['Category'].isin(top_categories)]

pivot_table = filtered_df.pivot_table(index='Date',
                                       columns='Category',
                                       values='Amount',
                                       aggfunc='sum',
                                       fill_value=0)

corr_matrix = pivot_table.corr()

fig = px.imshow(corr_matrix,
                text_auto=".2f",
                color_continuous_scale='RdBu_r',
                title="Correlation Heatmap of Top 10 Categories")

fig.update_layout(
    xaxis_title="Category",
    yaxis_title="Category",
    width=800,
    height=700
)

fig.show()


In [23]:
import plotly.express as px

#  Treemap for Total Amount per Category (small ones grouped)
category_amount = df.groupby('Category')['Amount'].sum().reset_index()
category_amount['Category'] = category_amount.apply(
    lambda x: 'Rest Expenses' if x['Amount'] < 20000 else x['Category'], axis=1
)
category_amount = category_amount.groupby('Category')['Amount'].sum().reset_index()

fig = px.treemap(category_amount,
                 path=['Category'],
                 values='Amount',
                 title='Total Amount Spent per Category',
                 color='Amount',
                 color_continuous_scale='Blues')
fig.update_traces(textinfo="label+value")
fig.show()


#  Horizontal Bar for Income vs Expense (better small value visibility)
income_expense = df.groupby('Income/Expense')['Amount'].sum().reset_index()
fig = px.bar(income_expense,
             x='Amount', y='Income/Expense',
             orientation='h',
             title='Income vs Expense',
             color='Amount',
             color_continuous_scale='RdYlGn')
fig.show()


# Smoothed Line Chart for Transaction Trends Over Time
daily_trends = df.groupby(df['Date'].dt.date)['Amount'].sum().reset_index()
fig = px.line(daily_trends,
              x='Date', y='Amount',
              title='Transaction Trends Over Time',
              markers=True,
              line_shape='spline')
fig.update_traces(line=dict(width=2, color='#6b5b95'))
fig.show()


#  Log Scale Bar for Payment Mode (handles tiny values)
mode_amount = df.groupby('Mode')['Amount'].sum().reset_index()
fig = px.bar(mode_amount,
             x='Mode', y='Amount',
             title='Total Amount by Payment Mode (Log Scale)',
             color='Amount',
             color_continuous_scale='Viridis',
             log_y=True)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


In [24]:
#  Insights Summary
print('Total Transactions:', len(df))
print('Total Expense:', df[df['Income/Expense'] == 'Expense']['Amount'].sum())
print('Total Income:', df[df['Income/Expense'] == 'Income']['Amount'].sum())
print('Most Common Payment Mode:', df['Mode'].mode()[0])
print('Most Frequent Category:', df['Category'].mode()[0])

Total Transactions: 2057
Total Expense: 1356344.13
Total Income: 3042355.35
Most Common Payment Mode: Cash
Most Frequent Category: Food
